In [1]:
import sys
sys.path.insert(0, '../')

import pandas as pd
from sklearn import metrics
#from keras.wrappers.scikit_learn import KerasClassifier
from classification_plots import plot_confusion_matrix
import matplotlib.pyplot as plt
import os
from collections import Counter
import numpy as np
import scipy 
%matplotlib notebook

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import warnings
warnings.filterwarnings('ignore')

In [3]:
audio_dir = '../../ESC-50/audio/'
path_to_db='../../ESC-50/'

In [4]:
from torch.utils.data import Dataset, DataLoader
import librosa
fs = 44100
class audiotrainset(Dataset):
    def __init__(self):
        
        root = "./data3/"
        self.data_list = []
        self.label_list = []
        for root, dir, files in os.walk("./data3/"):
            for file in files:
                if file.find('.npy')!= -1:
#                     print(os.path.join(root, file))
#                     print(int(root[13:]))
                    self.data_list.append(os.path.join(root, file))
                    self.label_list.append(int(root[13:]))
        print(len(self.data_list), len(self.label_list))
        
    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, index):

        path = self.data_list[index]
        data = np.load(path,allow_pickle = True)
        data = np.expand_dims(data, axis=0)
        label= self.label_list[index] 

        label = np.asarray(label)
        
        return data, label,path
audiodataloader = audiotrainset()
data,label,path = audiodataloader.__getitem__(895)
print(path)

2000 2000
./data3/class29\2-94807-A-29.npy


In [5]:
## Extracts LPC coefficients ###
import numpy as np
import scipy as sp
import scipy.signal as sig

def lpc_ref(signal, order):
    """Compute the Linear Prediction Coefficients.
    Return the order + 1 LPC coefficients for the signal. c = lpc(x, k) will
    find the k+1 coefficients of a k order linear filter:
      xp[n] = -c[1] * x[n-2] - ... - c[k-1] * x[n-k-1]
    Such as the sum of the squared-error e[i] = xp[i] - x[i] is minimized.
    Parameters
    ----------
    signal: array_like
        input signal
    order : int
        LPC order (the output will have order + 1 items)
    Note
    ----
    This is just for reference, as it is using the direct inversion of the
    toeplitz matrix, which is really slow"""
    if signal.ndim > 1:
        raise ValueError("Array of rank > 1 not supported yet")
#     if order > signal.size:
#         raise ValueError("Input signal must have a lenght >= lpc order")

    if order > 0:
        p = order + 1
        r = np.zeros(p, signal.dtype)
        # Number of non zero values in autocorrelation one needs for p LPC
        # coefficients
        nx = np.min([p, signal.size])
        x = np.correlate(signal, signal, 'full')
        r[:nx] = x[signal.size-1:signal.size+order]
        phi = np.dot(sp.linalg.inv(sp.linalg.toeplitz(r[:-1])), -r[1:])
        return np.concatenate(([1.], phi))
    else:
        return np.ones(1, dtype = signal.dtype)

def levinson_1d(r, order):
    """Levinson-Durbin recursion, to efficiently solve symmetric linear systems
    with toeplitz structure.
    Arguments
    ---------
        r : array-like
            input array to invert (since the matrix is symmetric Toeplitz, the
            corresponding pxp matrix is defined by p items only). Generally the
            autocorrelation of the signal for linear prediction coefficients
            estimation. The first item must be a non zero real.
    Note
    ----
    This implementation is in python, hence unsuitable for any serious
    computation. Use it as educational and reference purpose only.
    Levinson is a well-known algorithm to solve the Hermitian toeplitz
    equation:
                       _          _
        -R[1] = R[0]   R[1]   ... R[p-1]    a[1]
         :      :      :          :      *  :
         :      :      :          _      *  :
        -R[p] = R[p-1] R[p-2] ... R[0]      a[p]
                       _
    with respect to a (  is the complex conjugate). Using the special symmetry
    in the matrix, the inversion can be done in O(p^2) instead of O(p^3).  
    """
    r = np.atleast_1d(r)
    if r.ndim > 1:
        raise ValueError("Only rank 1 are supported for now.")

    n = r.size
    if order > n - 1:
        raise ValueError("Order should be <= size-1")
    elif n < 1:
        raise ValueError("Cannot operate on empty array !")

    if not np.isreal(r[0]):
        raise ValueError("First item of input must be real.")
    elif not np.isfinite(1/r[0]):
        raise ValueError("First item should be != 0")

    # Estimated coefficients
    a = np.empty(order+1, r.dtype)
    # temporary array
    t = np.empty(order+1, r.dtype)
    # Reflection coefficients
    k = np.empty(order, r.dtype)

    a[0] = 1.
    e = r[0]

    for i in xrange(1, order+1):
        acc = r[i]
        for j in range(1, i):
            acc += a[j] * r[i-j]
        k[i-1] = -acc / e
        a[i] = k[i-1]

        for j in range(order):
            t[j] = a[j]

        for j in range(1, i):
            a[j] += k[i-1] * np.conj(t[i-j])

        e *= 1 - k[i-1] * np.conj(k[i-1])

    return a, e, k

In [6]:
######## Wavelet Decomposition #######
import pywt
def wavelet_decompose(y,wtype):
    wt = pywt.Wavelet(wtype)
    cA = pywt.wavedec(y,wt,level=2)[0]
    cA = np.asarray(cA)
    return cA

In [7]:
##### Stats from a signal #####
def get_stats(y):
    split_array = np.array_split(y,4)
    stats_vector = np.zeros(len(split_array)*2)
    offset = 0
    for idx,chunk in enumerate(split_array,0):
        stats_vector[offset] = np.mean(chunk)
        stats_vector[offset + 1] = np.var(chunk)
        offset = offset + 2
    stats_vector = np.asarray(stats_vector)
    return (stats_vector)
    

In [8]:
######### Extract MFCC Coefficients #########
def extract_mfcc(y,sr):
    mfcc_array = librosa.feature.mfcc(y,sr,n_mfcc=13)
    mfcc_features = []
    for i in range(13):
        mfcc_wav = wavelet_decompose(mfcc_array[i,:],'haar')
        lpc_coeffs = lpc_ref(mfcc_wav,5)[1:]
        mfcc_features.append(np.asarray(lpc_coeffs))
    
    
    final_coeffs = 0
    for coeff in mfcc_features:
        final_coeffs = np.concatenate((final_coeffs,coeff),axis = None)
    
    final_coeffs = final_coeffs[1:]
    
    final_coeffs = np.asarray(final_coeffs) 
    return (final_coeffs)

In [9]:
def feature_extraction(y):
    sr = 44100
    f1 = (lpc_ref(y,9)[1:])
    f2 = (get_stats(wavelet_decompose(y,'haar')))
    f3 = (extract_mfcc(y,sr))
    f4 = (get_stats(wavelet_decompose(y,'db4')))
    f5 = (get_stats(wavelet_decompose(y,'sym4')))
    local_features =  np.concatenate((f1,f2,f3,f4,f5),axis = None)
    return local_features

In [10]:
import torch.nn as nn
import torch.nn.functional as F
class SpectralNet(nn.Module):
    def __init__(self,):
        super(SpectralNet, self).__init__()
        self.fc1 = nn.Linear(98,200)
        self.fc2 = nn.Linear(200,200)
        self.fc3 = nn.Linear(200,50)
        self.softmax = nn.Softmax()
        self.batchnorm1 = nn.BatchNorm1d(200)
        self.batchnorm2 = nn.BatchNorm1d(200)

    def forward(self, x):
        x = self.fc1(x)
        x = self.batchnorm1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = self.batchnorm2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = self.softmax(x)
        return x

model = SpectralNet()
print (model)
model.cuda()    

SpectralNet(
  (fc1): Linear(in_features=98, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=50, bias=True)
  (softmax): Softmax(dim=None)
  (batchnorm1): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


SpectralNet(
  (fc1): Linear(in_features=98, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=50, bias=True)
  (softmax): Softmax(dim=None)
  (batchnorm1): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [13]:
import time
for fileList in os.walk(audio_dir):
    count = 0
    for fname in fileList[2]:
        t1 = time.time()
        count += 1
        name_array = fname.split('.')
        name = name_array[0]
        name_array = name_array[0].split('-')
        curr_class = name_array[-1]
        if(name != '' and curr_class != ''):
            fpath = audio_dir + fname
            (sig, rate) = librosa.load(fpath, sr=None, mono=True,  dtype=np.float32)
            features = feature_extraction(sig)

            save_dir = './data3/class' + str(curr_class) + '/'
            
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
            np.save(save_dir + name + '.npy',features)
        t2 = time.time()
        print(count, t2-t1)

1 4.15310525894165
2 3.4279520511627197
3 3.8079965114593506
4 3.490097761154175
5 3.721195936203003
6 3.376580238342285
7 3.454676389694214
8 3.645894765853882
9 3.3152709007263184
10 3.5187227725982666
11 3.440006971359253
12 3.455332040786743
13 3.4447991847991943
14 3.377889633178711
15 3.5352299213409424
16 3.659200668334961
17 3.5526299476623535
18 3.536112070083618
19 3.455874443054199
20 3.4798829555511475
21 3.5257883071899414
22 3.599493980407715
23 3.3531548976898193
24 3.4509224891662598
25 3.4638564586639404
26 3.3734514713287354
27 3.410682201385498
28 3.7841856479644775
29 3.3609964847564697
30 3.4532699584960938
31 3.827561378479004
32 3.5338923931121826
33 3.3192403316497803
34 3.4020957946777344
35 3.383803129196167
36 3.458883762359619
37 3.3490757942199707
38 3.489793062210083
39 3.4277842044830322
40 3.4369258880615234
41 3.6569206714630127
42 3.7673139572143555
43 3.3995869159698486
44 3.9233858585357666
45 4.580155372619629
46 3.529059410095215
47 3.7427563667297

734 3.366655111312866
735 3.626070261001587
736 3.5029797554016113
737 3.4509663581848145
738 3.4436118602752686
739 3.2753121852874756
740 3.6519527435302734
741 3.3930366039276123
742 3.4337058067321777
743 3.675386905670166
744 3.599931240081787
745 3.439077854156494
746 3.753098964691162
747 3.4730422496795654
748 3.44099760055542
749 3.6195390224456787
750 3.2973225116729736
751 3.6219770908355713
752 3.3781063556671143
753 3.4937117099761963
754 3.726881504058838
755 3.546159029006958
756 3.7660646438598633
757 3.7365899085998535
758 3.9639461040496826
759 3.765777587890625
760 3.766075611114502
761 3.531911611557007
762 3.701345205307007
763 3.836963176727295
764 3.8647494316101074
765 3.659696102142334
766 3.4076483249664307
767 5.593539237976074
768 4.035034418106079
769 3.8435139656066895
770 5.858055114746094
771 3.727626323699951
772 3.657766103744507
773 3.8972795009613037
774 4.7377965450286865
775 4.224146127700806
776 4.142393112182617
777 3.91401743888855
778 3.7776715

1445 4.766209840774536
1446 4.791991710662842
1447 4.75700831413269
1448 4.8683576583862305
1449 4.857078552246094
1450 6.540769577026367
1451 5.815582990646362
1452 4.934784650802612
1453 4.967979669570923
1454 4.757965803146362
1455 4.911267995834351
1456 4.95137095451355
1457 4.921654224395752
1458 4.818026304244995
1459 4.76144552230835
1460 4.905880689620972
1461 4.992854595184326
1462 4.884907245635986
1463 5.154206037521362
1464 5.2999043464660645
1465 5.269836187362671
1466 5.026568651199341
1467 4.864529848098755
1468 4.703273773193359
1469 4.8830578327178955
1470 5.160989284515381
1471 5.2352941036224365
1472 5.017249822616577
1473 5.039617538452148
1474 4.562953948974609
1475 4.86203145980835
1476 4.647244691848755
1477 4.715893745422363
1478 4.6258933544158936
1479 4.698102951049805
1480 4.584888935089111
1481 4.69238543510437
1482 4.69409966468811
1483 4.8098413944244385
1484 6.628084897994995
1485 5.197390079498291
1486 7.123199701309204
1487 6.06043553352356
1488 6.80743

In [15]:
import torch.optim as optim
import torch
from torch.utils.data.sampler import SubsetRandomSampler

device = 0
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

validation_split = .25
random_seed= 42
shuffle_dataset = True
dataset_size = 2000
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

dataset  = audiotrainset()
trainloader = torch.utils.data.DataLoader(dataset, batch_size=5, 
                                           sampler=train_sampler,pin_memory=True)
valloader = torch.utils.data.DataLoader(dataset, batch_size=5,
                                                sampler=valid_sampler,pin_memory=True)


for epoch in range(5000):  # loop over the dataset multiple times
    
    count = 0
    validation_accuracy = 0
    running_loss = 0.0
    training_accuracy = 0
    ######## training Stage ###########
    for i, data in enumerate(trainloader, 0):
        count += 1
        inputs, labels,paths = data
        inputs = np.reshape(inputs,(5,98))
        inputs = inputs.float().to(device)
        labels = labels.long().to(device)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        acc_train = torch.eq(preds, labels).float().mean()
        training_accuracy += acc_train.item()
        if i % 100 == 99:
            print(epoch, i, running_loss/100)
            running_loss = 0
        
    epoch_loss = running_loss / count
    print("epoch loss:", epoch, epoch_loss)
    print("train_accuracy:", training_accuracy/count)
    
# #             torch.save(net, 'toy_model.pt')
        
    
    print('Finished Training')    
    running_loss = 0.0
    count = 0
    for i, data in enumerate(valloader, 0):
        count += 1
        inputs, labels,paths = data
        inputs = np.reshape(inputs,(5,98))
        inputs = inputs.float().to(device)
        labels = labels.long().to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        acc_val = torch.eq(preds, labels).float().mean()
        validation_accuracy += acc_val.item()
        
    print("val_accuracy:", validation_accuracy/count)

2000 2000
0 99 3.7551210165023803
0 199 3.77072484254837
0 299 3.764070842266083
epoch loss: 0 0.0
train_accuracy: 0.18466666996479034
Finished Training
val_accuracy: 0.10400000169873237
1 99 3.755330457687378
1 199 3.804699339866638
1 299 3.7467886352539064
epoch loss: 1 0.0
train_accuracy: 0.17733333642284074
Finished Training
val_accuracy: 0.11200000181794166
2 99 3.785102286338806
2 199 3.7520526218414307
2 299 3.7689229369163515
epoch loss: 2 0.0
train_accuracy: 0.17933333655198416
Finished Training
val_accuracy: 0.11000000163912774
3 99 3.76382465839386
3 199 3.7597504472732544
3 299 3.7619282817840576
epoch loss: 3 0.0
train_accuracy: 0.18466666991511982
Finished Training
val_accuracy: 0.1140000019967556
4 99 3.7598895239830017
4 199 3.7471429967880248
4 299 3.7829019975662233
epoch loss: 4 0.0
train_accuracy: 0.18533333674073219
Finished Training
val_accuracy: 0.12200000211596489
5 99 3.7717334747314455
5 199 3.7655927658081056
5 299 3.7531953835487366
epoch loss: 5 0.0
train_a

45 199 3.7558069944381716
45 299 3.7626172161102294
epoch loss: 45 0.0
train_accuracy: 0.18600000341733297
Finished Training
val_accuracy: 0.09800000146031379
46 99 3.779059262275696
46 199 3.7458539986610413
46 299 3.7521158862113952
epoch loss: 46 0.0
train_accuracy: 0.18866667027274767
Finished Training
val_accuracy: 0.11400000184774399
47 99 3.771425840854645
47 199 3.7437555027008056
47 299 3.74721985578537
epoch loss: 47 0.0
train_accuracy: 0.19266667013367017
Finished Training
val_accuracy: 0.10800000205636025
48 99 3.764294216632843
48 199 3.7843300986289976
48 299 3.7391803240776063
epoch loss: 48 0.0
train_accuracy: 0.18466667026281358
Finished Training
val_accuracy: 0.11000000163912774
49 99 3.741815330982208
49 199 3.772375795841217
49 299 3.7582030129432678
epoch loss: 49 0.0
train_accuracy: 0.1906666701535384
Finished Training
val_accuracy: 0.11400000169873238
50 99 3.752510404586792
50 199 3.7775511264801027
50 299 3.7550059247016905
epoch loss: 50 0.0
train_accuracy: 0.

90 199 3.767034981250763
90 299 3.747001848220825
epoch loss: 90 0.0
train_accuracy: 0.18466667011380195
Finished Training
val_accuracy: 0.10000000149011612
91 99 3.7634763813018797
91 199 3.7677495098114013
91 299 3.7630945873260497
epoch loss: 91 0.0
train_accuracy: 0.18533333669106167
Finished Training
val_accuracy: 0.10400000169873237
92 99 3.76564945936203
92 199 3.7442554235458374
92 299 3.763270297050476
epoch loss: 92 0.0
train_accuracy: 0.1886666699250539
Finished Training
val_accuracy: 0.11200000181794166
93 99 3.7535642051696776
93 199 3.7463969874382017
93 299 3.7730945920944214
epoch loss: 93 0.0
train_accuracy: 0.19133333683013917
Finished Training
val_accuracy: 0.12000000178813934
94 99 3.7533467912673952
94 199 3.768545434474945
94 299 3.747436864376068
epoch loss: 94 0.0
train_accuracy: 0.190000003327926
Finished Training
val_accuracy: 0.1160000018775463
95 99 3.7563499093055723
95 199 3.749128668308258
95 299 3.759294605255127
epoch loss: 95 0.0
train_accuracy: 0.1913

134 299 3.7551189255714417
epoch loss: 134 0.0
train_accuracy: 0.18000000312924386
Finished Training
val_accuracy: 0.09600000157952308
135 99 3.759607980251312
135 199 3.769502115249634
135 299 3.7602272963523866
epoch loss: 135 0.0
train_accuracy: 0.1846666701634725
Finished Training
val_accuracy: 0.11000000193715095
136 99 3.780769371986389
136 199 3.7545077085494993
136 299 3.7453960299491884
epoch loss: 136 0.0
train_accuracy: 0.18733333667119345
Finished Training
val_accuracy: 0.10200000151991845
137 99 3.749708867073059
137 199 3.76531188249588
137 299 3.7671523189544676
epoch loss: 137 0.0
train_accuracy: 0.18866667012373606
Finished Training
val_accuracy: 0.11200000166893005
138 99 3.7357017731666566
138 199 3.7882369351387024
138 299 3.7620307087898253
epoch loss: 138 0.0
train_accuracy: 0.18600000326832136
Finished Training
val_accuracy: 0.10400000154972076
139 99 3.761871757507324
139 199 3.7610280108451843
139 299 3.761932134628296
epoch loss: 139 0.0
train_accuracy: 0.1833

178 199 3.7640306687355043
178 299 3.7655184674263
epoch loss: 178 0.0
train_accuracy: 0.18800000349680582
Finished Training
val_accuracy: 0.10600000157952309
179 99 3.7796241211891175
179 199 3.723064124584198
179 299 3.7752889943122865
epoch loss: 179 0.0
train_accuracy: 0.18800000349680582
Finished Training
val_accuracy: 0.1160000018775463
180 99 3.7645141291618347
180 199 3.7584663009643555
180 299 3.7501109647750854
epoch loss: 180 0.0
train_accuracy: 0.18866666987538339
Finished Training
val_accuracy: 0.10800000190734864
181 99 3.750755367279053
181 199 3.7595269584655764
181 299 3.762398355007172
epoch loss: 181 0.0
train_accuracy: 0.18933333704868952
Finished Training
val_accuracy: 0.11000000178813935
182 99 3.758619575500488
182 199 3.7361247634887693
182 299 3.7706342244148257
epoch loss: 182 0.0
train_accuracy: 0.19333333685994147
Finished Training
val_accuracy: 0.11000000163912774
183 99 3.735098669528961
183 199 3.770977931022644
183 299 3.7575133442878723
epoch loss: 183 

...

val_accuracy: 0.13000000193715094
4803 99 3.6982960057258607
4803 199 3.664428963661194
4803 299 3.679228367805481
epoch loss: 4803 0.0
train_accuracy: 0.2646666719019413
Finished Training
val_accuracy: 0.11800000190734863
4804 99 3.7021205163002016
4804 199 3.6758359122276305
4804 299 3.689226574897766
epoch loss: 4804 0.0
train_accuracy: 0.2566666716833909
Finished Training
val_accuracy: 0.12400000214576722
4805 99 3.6708751487731934
4805 199 3.661574726104736
4805 299 3.715805993080139
epoch loss: 4805 0.0
train_accuracy: 0.2633333388964335
Finished Training
val_accuracy: 0.13400000244379043
4806 99 3.6968343329429625
4806 199 3.6782299137115477
4806 299 3.6570329165458677
epoch loss: 4806 0.0
train_accuracy: 0.27066667209068934
Finished Training
val_accuracy: 0.12400000214576722
4807 99 3.6699169659614563
4807 199 3.689804480075836
4807 299 3.6693300104141233
epoch loss: 4807 0.0
train_accuracy: 0.2713333391646544
Finished Training
val_accuracy: 0.12000000178813934
4808 99 3.696378

val_accuracy: 0.13200000241398813
4846 99 3.6881261348724363
4846 199 3.6788754057884216
4846 299 3.6775041937828066
epoch loss: 4846 0.0
train_accuracy: 0.2646666717529297
Finished Training
val_accuracy: 0.13800000235438348
4847 99 3.6795801973342894
4847 199 3.674867126941681
4847 299 3.6832739973068236
epoch loss: 4847 0.0
train_accuracy: 0.2680000053346157
Finished Training
val_accuracy: 0.13200000196695327
4848 99 3.678485884666443
4848 199 3.693622829914093
4848 299 3.6590840578079225
epoch loss: 4848 0.0
train_accuracy: 0.2680000050365925
Finished Training
val_accuracy: 0.13200000196695327
4849 99 3.685167727470398
4849 199 3.6778755044937133
4849 299 3.676384406089783
epoch loss: 4849 0.0
train_accuracy: 0.26466667249798775
Finished Training
val_accuracy: 0.1300000022351742
4850 99 3.7082226157188414
4850 199 3.6401101112365724
4850 299 3.6865693044662478
epoch loss: 4850 0.0
train_accuracy: 0.2700000056127707
Finished Training
val_accuracy: 0.1260000018775463
4851 99 3.6844470

4889 99 3.6940371656417845
4889 199 3.686173417568207
4889 299 3.6849203944206237
epoch loss: 4889 0.0
train_accuracy: 0.25800000528494516
Finished Training
val_accuracy: 0.12200000196695328
4890 99 3.6527081418037413
4890 199 3.6734409880638124
4890 299 3.7028021907806394
epoch loss: 4890 0.0
train_accuracy: 0.27133333931366604
Finished Training
val_accuracy: 0.1300000023841858
4891 99 3.6648600482940674
4891 199 3.6801403284072878
4891 299 3.702467918395996
epoch loss: 4891 0.0
train_accuracy: 0.26266667197148
Finished Training
val_accuracy: 0.1220000022649765
4892 99 3.6741265773773195
4892 199 3.685096750259399
4892 299 3.6870143365859986
epoch loss: 4892 0.0
train_accuracy: 0.2633333385984103
Finished Training
val_accuracy: 0.1300000022351742
4893 99 3.6664211201667785
4893 199 3.701758394241333
4893 299 3.676389682292938
epoch loss: 4893 0.0
train_accuracy: 0.26733333835999173
Finished Training
val_accuracy: 0.1320000022649765
4894 99 3.699621126651764
4894 199 3.6865025997161864

4932 199 3.6886017727851867
4932 299 3.6686985540390014
epoch loss: 4932 0.0
train_accuracy: 0.2673333386083444
Finished Training
val_accuracy: 0.12000000223517418
4933 99 3.6691997480392455
4933 199 3.693669834136963
4933 299 3.6832668352127076
epoch loss: 4933 0.0
train_accuracy: 0.2633333388964335
Finished Training
val_accuracy: 0.1320000022649765
4934 99 3.6780167412757874
4934 199 3.679066812992096
4934 299 3.6630846953392027
epoch loss: 4934 0.0
train_accuracy: 0.2733333390454451
Finished Training
val_accuracy: 0.14000000238418578
4935 99 3.6716010642051695
4935 199 3.656803231239319
4935 299 3.7132512617111204
epoch loss: 4935 0.0
train_accuracy: 0.26400000592072803
Finished Training
val_accuracy: 0.12800000205636025
4936 99 3.7032968688011167
4936 199 3.6681509137153627
4936 299 3.677844750881195
epoch loss: 4936 0.0
train_accuracy: 0.2640000055233637
Finished Training
val_accuracy: 0.11800000190734863
4937 99 3.6726550459861755
4937 199 3.6944836139678956
4937 299 3.6838230609

4975 299 3.6773740696907042
epoch loss: 4975 0.0
train_accuracy: 0.26933333863814674
Finished Training
val_accuracy: 0.1240000019967556
4976 99 3.6915328669548035
4976 199 3.669498336315155
4976 299 3.671652326583862
epoch loss: 4976 0.0
train_accuracy: 0.26733333835999173
Finished Training
val_accuracy: 0.12200000181794167
4977 99 3.6667283630371093
4977 199 3.659328284263611
4977 299 3.6868589425086977
epoch loss: 4977 0.0
train_accuracy: 0.2760000052054723
Finished Training
val_accuracy: 0.1160000018775463
4978 99 3.6956503653526305
4978 199 3.6649520015716552
4978 299 3.6899083709716796
epoch loss: 4978 0.0
train_accuracy: 0.26333333854873975
Finished Training
val_accuracy: 0.12400000214576722
4979 99 3.668296904563904
4979 199 3.6987877106666565
4979 299 3.6651689410209656
epoch loss: 4979 0.0
train_accuracy: 0.2713333391646544
Finished Training
val_accuracy: 0.10200000166893006
4980 99 3.6828999090194703
4980 199 3.6790325808525086
4980 299 3.6848258113861085
epoch loss: 4980 0.0